
agrupar por cidade
agrupar por ano count total por cidade
valor de univercidade
valor total de univercidade
col: distric code
county(condado):ano -> col:awardee(univercidades(gerar contador))-> toltal recebido

Mantém a D_50 (é redundante, mas deixa la)
Cria d_59 que vai ter tudo 0 ai quando agregarmos ela vai receber o valor de quantas vezes uma mesma universidade apareceu e vc pode usar a coluna d_50 para contar (se quiser)
e uma outra coluna d_99 que vai receber também 0 no começo e quando agregar ela fica com 1,2,3,4... conforme aparecer a mesma universidade mais vezes com valores acima de 50k
No fim temos 3 colunas
d_50 que é uma dummy mesmo
d_59 que tem quantas vezes uma universidade apareceu por ano com valor de 50k, 1,2,3,4...
d_99 que tem quantas vezes uma universidade apareceu por ano com valor ACIMA de 50k, 1,2,3,4...

In [115]:
import pandas as pd
import numpy as np
from pathlib import Path

#path append
#base = Path('C:/workspace/nsf-award-extract') # Vitor's PC
base = Path('C:/collaboration/vitor/NSF_DATA') # Server

nsf=pd.read_csv(base/ 'allNSF1.csv',index_col = False) #Change this to the clean data later
#dft=pd.read_csv(base/ 'test.csv',index_col = False) #Change this to the clean data later

C:\Users\fernav4\AppData\Local\Temp\ipykernel_19212\3674381822.py:9: DtypeWarning: Columns (12,13,17,25,32,37,45,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nsf=pd.read_csv(base/ 'allNSF1.csv',index_col = False) #Change this to the clean data later


In [116]:
#Just checking if we have the same amount of register in each year as in this link https://www.nsf.gov/awardsearch/download.jsp
#nsf['year'] = pd.to_datetime(nsf['date'], format='%m/%d/%Y').dt.year

In [117]:
nsf.columns


Index(['Column', 'abstractText', 'agency', 'awardAgencyCode', 'awardee',
       'awardeeAddress', 'awardeeCity', 'awardeeCountryCode', 'awardeeCounty',
       'awardeeDistrictCode', 'awardeeName', 'awardeeStateCode',
       'awardeeZipCode', 'cfdaNumber', 'month_date', 'day_date', 'Year_date',
       'dunsNumber', 'estimatedTotalAmt', 'expDate 1', 'expDate 2',
       'expDate 3', 'fundAgencyCode', 'fundProgramName', 'fundsObligatedAmt',
       'id', 'parentDunsNumber', 'pdPIName', 'perfAddress', 'perfCity',
       'perfCountryCode', 'perfLocation', 'perfZipCode', 'piEmail',
       'piFirstName', 'piLastName', 'piMiddeInitial', 'piPhone', 'poEmail',
       'startDate', 'title', 'transType', 'perfCounty', 'perfDistrictCode',
       'perfStateCode', 'primaryProgram', 'coPDPI', 'poName', 'poPhone',
       'publicationResearch', 'publicationConference',
       'projectOutComesReport'],
      dtype='object')

Filling the missing county

In [118]:
mapping = nsf.groupby('awardee')['awardeeCounty'].first().to_dict()
nsf['awardeeCounty'] = nsf['awardeeCounty'].fillna(nsf['awardee'].map(mapping))

Keeping only the funding with fundProgramName specified 

In [141]:
nsf = nsf[~nsf['fundProgramName'].isna()]

# Extracting the Universities that has missing county
We need to use this list to find the county and later use the filled file to map this location back to the main file

In [142]:
nsf[nsf['awardeeCounty'].isna()]

,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport
8,7,Continuous aseptic processing and packaging is...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,38,The Trek of the Mammoth is a three-week multid...,NSF,4900.0,CHADRON STATE COLLEGE,10th and Main Streets,chadron,US,None,NE03,...,NaN,NaN,NaN,NaN,Ronald R Weedon,NaN,NaN,NaN,NaN,NaN
385,55,This three-year project is designed to improve...,NSF,4900.0,MCNEESE STATE UNIVERSITY,Box 90655,Lake Charles,US,None,LA07,...,NaN,NaN,NaN,NaN,Gary D Kratzer,NaN,NaN,NaN,NaN,NaN
924,7,PROJECT PTRA-PLUS is a program of the American...,NSF,4900.0,AMERICAN ASSOCIATION OF PHYSICS TEACHERS INC,One Physics Ellipse,College Park,US,None,MD05,...,NaN,MD05,MD,"490106 NSF, EDUCATION & HUMAN RESOURC",James H Nelson,NaN,NaN,NaN,NaN,NaN
993,76,The Bermuda Biological Station will receive su...,NSF,4900.0,BERMUDA INSTITUTE OF OCEAN SCIENCES (BIOS) INC.,17 Biological Station,St. George's GE01,BD,None,NaN,...,NaN,NaN,NaN,490100 NSF RESEARCH & RELATED ACTIVIT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191110,11257,New cryptographic systems can improve the secu...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,RALEIGH,NC02,NC,NaN,NaN,Sankar Basu,7.032928e+09,NaN,NaN,NaN
191356,11503,The U.S. Southwest is a drought-prone region t...,NSF,4900.0,NaN,NaN,Ann Arbor,US,NaN,MI,...,DAVIS,CA04,CA,NaN,NaN,Soumaya Belmecheri,7.032929e+09,"Macarewich, Sophia I. and Poulsen, Christopher...",NaN,Water resource use in the southwestern U.S. (h...
191433,11580,Extreme weather events (heat waves and cold sn...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,Raleigh,NC02,NC,NaN,NaN,Bruce Hamilton,7.032920e+09,NaN,NaN,NaN
191476,11623,This award is funded in whole or in part under...,NSF,4900.0,NORTH CAROLINA STATE UNIVERSITY,2601 Wolf Village Way,Raleigh,US,None,NC04,...,RALEIGH,NC02,NC,NaN,NaN,Bianca Garner,7.032928e+09,NaN,NaN,NaN


In [143]:
filtered_nsf = nsf[nsf['awardeeCounty'].isna()].drop_duplicates(subset='awardee')
filtered_nsf.to_csv(base/ "missing_NSF.csv", index = False)


Code for filling the missing value should come here

191555

In [111]:
# missing_values = dfp[dfp['awardeeCounty'].isna()]
# for index,row in missing_values.iterrows():
#   key = row["awardeeName"]
#   mat = dfc[dfc['awardeeName']== key]
#   if mat.shape[0]>0:
#     dfp.at[index, dfp.columns[1]] = mat.iloc[0][dfc.columns[8]]
# mat.shape
#dfp.to_csv("I-corp-Full.csv") #for unknown reason we still have 1 missing county "Chautauqua County" (manually imputed)

In [ ]:
# df1 = df[['awardeeName', 'awardeeCounty']]
# df2 = dfc[['awardeeName', 'awardeeCounty']]
# dfc.columns[8]

In [ ]:
# pd.set_option('display.max_columns', None)
# df=pd.read_csv('genICorp.csv') 
# dfp=pd.read_csv('Panel2.csv') #using the already organized data
# dfc = pd.read_csv("I-Corp_county.csv") # This is the file I manually updated the county column

# #File to create the panel
# dfp = pd.read_csv("I-corp-Full.csv")

Creating the file with no missing values (later we don't need this because we will have all the Universities with their respective county)

In [145]:
nsf_all = nsf[~nsf['awardeeCounty'].isna()] #187755 + 3773 (correct)

(187755, 52)

Creating the first panel

In [147]:
def cross_to_panel(df):
    df = df.set_index(['Year_date','awardeeCounty'])
    df = df.sort_index()
    return df
panel_data = cross_to_panel(nsf_all)
#panel_data.to_csv('panel2.csv')

# def cross_to_panel(df):
#     df['date'] = pd.to_datetime(df['Year_date'], format='%Y')
#     df['year'] = df['date'].dt.year
#     df = df.set_index(['year', 'awardeeCounty'])
#     return df

# # Example usage

# panel_data = cross_to_panel(df)
# panel_data.to_csv('cross2.csv')
#panel_data

Creating the Panel with aggregation (I think we don't need this)

In [148]:
#grouped_multiple = df.groupby(['Year_date', 'awardeeCounty'])
#grouped_multiple = grouped_multiple.reset_index()
#print(grouped_multiple)

# p_table = pd.pivot_table(nsf_all, index = ['Year_date',"awardeeCounty"])
# p_table.to_csv("panel_nsf_all.csv")
# p_table

# Repeating the same process we did for I-Corps 

In [149]:
data = nsf_all.copy()

Creating Filters to get the correct programs

In [155]:
programs = data.fundProgramName.unique()
len(programs)

1183

In [162]:
df_corps = data[data['fundProgramName'] == 'I-Corps']

df_SBIR_1 = data[data['fundProgramName'] == 'SBIR Phase I']
df_SBIR_2 = data[data['fundProgramName'] == 'SBIR Phase II']
# df_corps = data[data['fundProgramName'] == 'I-Corps']
# df_corps = data[data['fundProgramName'] == 'I-Corps']


,Column,abstractText,agency,awardAgencyCode,awardee,awardeeAddress,awardeeCity,awardeeCountryCode,awardeeCounty,awardeeDistrictCode,...,perfCounty,perfDistrictCode,perfStateCode,primaryProgram,coPDPI,poName,poPhone,publicationResearch,publicationConference,projectOutComesReport
11262,224,9360320 Spooner This project is funded under ...,NSF,4900.0,MISSION RESEARCH CORPORATION,735 State Street,Santa Barbara,US,Santa Barbara,CA24,...,Santa Barbara,CA24,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11288,250,9360294 Abramson This project is funded under...,NSF,4900.0,OUSD(AT&L),P.O. Box 1838,Honolulu,US,Honolulu,HI01,...,Honolulu,HI01,HI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,251,WPC 2 B J ...,NSF,4900.0,OUSD(AT&L),2402 Clover Basin Drive,LONGMONT,US,Longmont,CO04,...,Longmont,CO04,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11295,257,9361690 Jacobsmeyer This project is funded un...,NSF,4900.0,OUSD(AT&L),P O Box 50378,Colorado Springs,US,Colorado Springs,CO05,...,Colorado Springs,CO05,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11854,816,A method is proposed to enable full-energy inj...,NSF,4900.0,"DULY RESEARCH, INC.",1912 MacArthur Street,Rancho Palos Verdes,US,Rancho Palos Verdes,CA33,...,Rancho Palos Verdes,CA33,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191395,11542,The broader impact /commercial potential of th...,NSF,4900.0,"INNOVISION, LLC.","5600 KENTSHIRE DRIVE, UNIT 5",DAYTON,US,Dayton,OH10,...,DAYTON,OH10,OH,NaN,NaN,Alastair Monk,7.032924e+09,NaN,NaN,Scoliosis is an abnormal curvature of the spin...
191398,11545,The broader impact of this Small Business Inno...,NSF,4900.0,"CIPHERMODE LABS, INC.",445 S FIGUEROA ST STE 3190,LOS ANGELES,US,Los Angeles,CA,...,LOS ANGELES,CA37,CA,NaN,NaN,Peter Atherton,7.032929e+09,NaN,NaN,This project proposes a new software platform ...
191491,11638,The broader impact /commercial potential of th...,NSF,4900.0,SURGIVANCE INC.,310 E 67TH ST,NEW YORK,US,New York,NY12,...,NEW YORK,NY12,NY,NaN,NaN,Alastair Monk,7.032924e+09,NaN,NaN,NaN
191535,11682,The broader impact/commercial potential of thi...,NSF,4900.0,OPEN SOURCE INSTRUMENTS INC.,22 BEDFORD ST,WALTHAM,US,Waltham,MA05,...,WATERTOWN,MA05,MA,NaN,NaN,Anna Brady,7.032927e+09,NaN,NaN,When we add up the masses of the galaxies that...


In [159]:
# pd.set_option('display.max_columns', None)
# df=pd.read_csv('genICorp.csv') 
# dfp=pd.read_csv('Panel2.csv') #using the already organized data
# dfc = pd.read_csv("I-Corp_county.csv") # This is the file I manually updated the county column

# #File to create the panel
# dfp = pd.read_csv("I-corp-Full.csv")

In [ ]:
# pd.set_option('display.max_columns', None)
# df=pd.read_csv('genICorp.csv') 
# dfp=pd.read_csv('Panel2.csv') #using the already organized data
# dfc = pd.read_csv("I-Corp_county.csv") # This is the file I manually updated the county column

# #File to create the panel
# dfp = pd.read_csv("I-corp-Full.csv")

In [127]:
data['d_inv'] = data.apply(lambda x: 0 if x.estimatedTotalAmt <= 50000.0 else 1,axis = 1)# cria variavel categorica para valores iguais ou menores que 50.000 e maiores que 50.000
data['d_59'] = 1
data['d_99'] = 0
data['estimatedTotalAmt_d_99'] = 0

In [128]:
df_clear = data.groupby(['awardeeCounty','Year_date','awardee','d_inv'])[['d_59','estimatedTotalAmt','d_99','estimatedTotalAmt_d_99']].sum()# agrupando por county
df_clear = df_clear.reset_index()

In [129]:
df_below50 = df_clear[df_clear['d_inv'] == 0].reset_index()
df_below50 = df_below50.drop(columns =['index'])


In [130]:
df_over50 = df_clear[df_clear['d_inv'] == 1].reset_index()
df_over50['d_99'] = df_over50['d_59']
df_over50['d_59'] = 0 
df_over50['estimatedTotalAmt_d_99'] = df_over50['estimatedTotalAmt']
df_over50['estimatedTotalAmt'] = 0
df_over50 = df_over50.drop(columns = ['index'])



In [131]:
for index,row in df_below50.iterrows():
  key = row['awardee']
  met = df_over50[df_over50['awardee'] == key]
  key_year = row['Year_date']
  met_year = met[met['Year_date'] == key_year]
  if met_year.shape[0]>0:
    df_below50.at[index,'d_99'] = met_year.iloc[0]['d_99']
    df_below50.at[index,'estimatedTotalAmt_d_99'] = met_year.iloc[0]['estimatedTotalAmt_d_99']
    df_over50 = df_over50.drop(met_year.index[0])

In [132]:
df_below50 = df_below50.merge(df_over50,how = 'outer')
df_below50['total'] = df_below50.apply(lambda x: x['estimatedTotalAmt'] + x['estimatedTotalAmt_d_99'],axis = 1)

In [133]:
df_final = df_below50.sort_values(by = ['awardeeCounty','Year_date'])
df_final = df_final.reset_index(inplace = False).drop(columns = ['index'])

Saving the file

In [134]:
df_final
df_final.to_csv("nsf_all.csv", index=False)


In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! abaixo so testes !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
x = data[data['awardee']== 'REGENTS OF THE UNIVERSITY OF CALIFORNIA, THE']
len(x[x['Year_date']== 2017])

In [ ]:
df_below50[df_below50['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_below50[df_below50['awardee'] == 'THE UNIVERSITY OF AKRON']

In [ ]:
df_over50[df_over50['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_final[df_final['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
#df_melt = pd.melt(data,id_vars= ['Year_date','awardee','estimatedTotalAmt'], value_vars = ['d_inv'] )
#df_melt

In [ ]:
data[data['awardee'] == 'NATIONAL COLLEGIATE INVENTORS & INNOVATORS ALLIANCE, INC.']

In [ ]:
df_50 = df_50.groupby(['Year_date','awardee'])[['estimatedTotalAmt']].sum()# agrupando por county

In [ ]:
df_50 = df_50.reset_index( )


In [ ]:
SELECT * FROM `post` WHERE source_id >= 175 AND created_date < '2023-01-01'
85840

DELETE epecificar a tabela para apagar os dados FROM `insight` INNER JOIN `post` on insight.post_id = post.id WHERE post.source_id = 175 AND post.created_date < '2023-01-01'; 
 numeros de registros apagados 13377784

WITH postIds AS (
 SELECT id FROM post WHERE source_id >= 175 AND created_date < '2023-01-01'
)

DELETE * 
FROM insight
WHERE post_id IN postIds

In [ ]:
79+1125

In [ ]:
df_over50.shape


In [ ]:
x = 0 
for index,row in df_below50.iterrows():
  key = row['awardee']
  met = df_over50[df_over50['awardee'] == key]
  key_year = row['Year_date']
  met_year = met[met['Year_date'] == key_year]
  if met_year.shape[0] > 0:
    x+=1
    print(met_year.index[0])
print(x)

In [ ]:
dif = df_over50[df_over50['awardee'] != key]

In [ ]:
x = df_over50['awardee'] != df_below50['awardee']

In [ ]:
dif

,index,awardeeCounty,Year_date,awardee,d_inv,estimatedTotalAmt,d_59,d_99
0,1,Akron,2013,THE UNIVERSITY OF AKRON,1,300000,1,0
1,6,Akron,2017,THE UNIVERSITY OF AKRON,1,300000,1,0
2,18,Albuquerque,2017,"UNIVERSITY OF NEW MEXICO, THE",1,443631,1,0
3,22,Alexandria,2019,GEM CONSORTIUM,1,3498254,1,0
4,29,Ames,2017,IOWA STATE UNIVERSITY OF SCIENCE AND TECHNOLOGY,1,498750,1,0
...,...,...,...,...,...,...,...,...
163,1254,Washington,2017,"GEORGE WASHINGTON UNIVERSITY, THE",1,499502,1,0
164,1265,West Lafayette,2015,PURDUE UNIVERSITY,1,197280,1,0
165,1269,West Lafayette,2018,PURDUE UNIVERSITY,1,160000,1,0
166,1275,Wichita,2016,WICHITA STATE UNIVERSITY,1,200000,1,0


In [ ]:
df_test = data.groupby(['awardeeCounty','Year_date','awardee'])[['estimatedTotalAmt']].sum()# agrupando por county

In [ ]:
df_test

In [ ]:
#min_value = data.estimatedTotalAmt.min() # menor valor da coluna cfdaNumber
df_50['n_invest'] = df_50.apply(lambda x: x.estimatedTotalAmt/min_value if x.estimatedTotalAmt> min_value else min_value/min_value, axis=1)# add col n

In [ ]:
df_test.reset_index(inplace =True)
df_test.head(4)


In [ ]:
for index, row in df_test.iterrows():
  key = row['awardee']
  met = data[data['awardee']==key]
  

    